In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import urllib.request
from io import StringIO
import requests
#import missingno as msno

# matplotlib
from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

# scipy specifics
from scipy import stats as sps
from scipy.interpolate import interp1d

In [5]:
try:
    from urllib.request import Request, urlopen  # Python 3
except ImportError:
    from urllib2 import Request, urlopen  # Python 2

req = Request('https://cloud.minsa.gob.pe/s/AC2adyLkHCKjmfm/download')
req.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0')
content = urlopen(req)

test = pd.read_csv(content, sep=";", na_values='EN INVESTIGACIÓN', encoding='utf-8')
test

HTTPError: HTTP Error 503: Service Temporarily Unavailable

In [6]:
content= "positivos_covid.csv"

test = pd.read_csv(content, sep=";", na_values='EN INVESTIGACIÓN', encoding='utf-8')
test

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
0,20220311,JUNIN,HUANCAYO,EL TAMBO,AG,68.0,FEMENINO,20210414.0,120114.0,24615058.0
1,20220311,LIMA,LIMA,SAN JUAN DE MIRAFLORES,AG,21.0,MASCULINO,20210413.0,150133.0,24731649.0
2,20220311,AREQUIPA,AREQUIPA,PAUCARPATA,AG,27.0,MASCULINO,20220222.0,40112.0,24731664.0
3,20220311,PIURA,PIURA,CASTILLA,AG,20.0,MASCULINO,20220114.0,200104.0,24731665.0
4,20220311,ANCASH,SANTA,CHIMBOTE,PR,23.0,MASCULINO,20200704.0,21801.0,24856166.0
...,...,...,...,...,...,...,...,...,...,...
3535720,20220311,LIMA,LIMA,LIMA,PCR,23.0,FEMENINO,20220108.0,150101.0,NaN
3535721,20220311,LIMA,LIMA,RIMAC,PR,51.0,FEMENINO,20201002.0,150128.0,NaN
3535722,20220311,TACNA,TACNA,CALANA,PCR,37.0,FEMENINO,20210529.0,230103.0,NaN
3535723,20220311,LIMA,LIMA,CHORRILLOS,PCR,68.0,FEMENINO,20210324.0,150108.0,NaN


In [7]:
test['FECHA_RESULTADO'] = pd.to_datetime(test['FECHA_RESULTADO'].astype(str), format='%Y%m%d')
test.tail()

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
3535720,20220311,LIMA,LIMA,LIMA,PCR,23.0,FEMENINO,2022-01-08,150101.0,NaN
3535721,20220311,LIMA,LIMA,RIMAC,PR,51.0,FEMENINO,2020-10-02,150128.0,NaN
3535722,20220311,TACNA,TACNA,CALANA,PCR,37.0,FEMENINO,2021-05-29,230103.0,NaN
3535723,20220311,LIMA,LIMA,CHORRILLOS,PCR,68.0,FEMENINO,2021-03-24,150108.0,NaN
3535724,20220311,JUNIN,CHANCHAMAYO,PICHANAQUI,AG,0.0,MASCULINO,2022-02-09,120303.0,NaN


In [8]:
test.isnull().sum()

FECHA_CORTE             0
DEPARTAMENTO          321
PROVINCIA          168286
DISTRITO           168286
METODODX                0
EDAD                  347
SEXO                    1
FECHA_RESULTADO      2023
UBIGEO             168286
id_persona          67284
dtype: int64

In [9]:
indice_departamento = pd.read_csv('https://raw.githubusercontent.com/annaabsi/git-scraper-covid19/main/resultados/positivos_por_departamentos.csv')
indice_departamento

,DEPARTAMENTO,METODODX,POBLACION,INDICE
0,AMAZONAS,43766,426806,10254
1,ANCASH,124901,1180638,10579
2,APURIMAC,38647,430736,8972
3,AREQUIPA,211730,1497438,14139
4,AYACUCHO,47943,668213,7175
5,CAJAMARCA,95474,1453711,6568
6,CALLAO,146169,1129854,12937
7,CUSCO,116318,1357075,8571
8,HUANCAVELICA,25538,365317,6991
9,HUANUCO,51881,760267,6824


In [10]:
test = pd.merge(test, indice_departamento,
                        how="left", on=["DEPARTAMENTO"])
test

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
0,20220311,JUNIN,HUANCAYO,EL TAMBO,AG,68.0,FEMENINO,2021-04-14,120114.0,24615058.0,125271.0,1361467.0,9201.0
1,20220311,LIMA,LIMA,SAN JUAN DE MIRAFLORES,AG,21.0,MASCULINO,2021-04-13,150133.0,24731649.0,1600339.0,10628470.0,15057.0
2,20220311,AREQUIPA,AREQUIPA,PAUCARPATA,AG,27.0,MASCULINO,2022-02-22,40112.0,24731664.0,211730.0,1497438.0,14139.0
3,20220311,PIURA,PIURA,CASTILLA,AG,20.0,MASCULINO,2022-01-14,200104.0,24731665.0,157210.0,2047954.0,7676.0
4,20220311,ANCASH,SANTA,CHIMBOTE,PR,23.0,MASCULINO,2020-07-04,21801.0,24856166.0,124901.0,1180638.0,10579.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535720,20220311,LIMA,LIMA,LIMA,PCR,23.0,FEMENINO,2022-01-08,150101.0,NaN,1600339.0,10628470.0,15057.0
3535721,20220311,LIMA,LIMA,RIMAC,PR,51.0,FEMENINO,2020-10-02,150128.0,NaN,1600339.0,10628470.0,15057.0
3535722,20220311,TACNA,TACNA,CALANA,PCR,37.0,FEMENINO,2021-05-29,230103.0,NaN,53615.0,370974.0,14452.0
3535723,20220311,LIMA,LIMA,CHORRILLOS,PCR,68.0,FEMENINO,2021-03-24,150108.0,NaN,1600339.0,10628470.0,15057.0


In [11]:
poblacion_csv = pd.read_csv('poblacion_provincia.csv')
poblacion_csv

,UBIGEO,PROVINCIA,POBLACION
0,10100,CHACHAPOYAS,63188
1,10200,BAGUA,84672
2,10300,BONGARA,26830
3,10400,CONDORCANQUI,51344
4,10500,LUYA,47827
...,...,...,...
191,240300,ZARUMILLA,56038
192,250100,CORONEL PORTILLO,447733
193,250200,ATALAYA,61049
194,250300,PADRE ABAD,77044


In [12]:
poblacion_dict = poblacion_csv.to_dict('split')
poblacion_dict['data']

[[10100, 'CHACHAPOYAS', 63188],
 [10200, 'BAGUA', 84672],
 [10300, 'BONGARA', 26830],
 [10400, 'CONDORCANQUI', 51344],
 [10500, 'LUYA', 47827],
 [10600, 'RODRIGUEZ DE MENDOZA', 33651],
 [10700, 'UTCUBAMBA', 119294],
 [20100, 'HUARAZ', 185276],
 [20200, 'AIJA', 6433],
 [20300, 'ANTONIO RAYMONDI', 13950],
 [20400, 'ASUNCION', 7710],
 [20500, 'BOLOGNESI', 24012],
 [20600, 'CARHUAZ', 50007],
 [20700, 'CARLOS FERMIN FITZCARRALD', 18496],
 [20800, 'CASMA', 57256],
 [20900, 'CORONGO', 8017],
 [21000, 'HUARI', 63264],
 [21100, 'HUARMEY', 33066],
 [21200, 'HUAYLAS', 56557],
 [21300, 'MARISCAL LUZURIAGA', 21787],
 [21400, 'OCROS', 7224],
 [21500, 'PALLASCA', 24371],
 [21600, 'POMABAMBA', 26675],
 [21700, 'RECUAY', 18085],
 [21800, 'SANTA', 474053],
 [21900, 'SIHUAS', 28630],
 [22000, 'YUNGAY', 55769],
 [30100, 'ABANCAY', 120116],
 [30200, 'ANDAHUAYLAS', 150758],
 [30300, 'ANTABAMBA', 11781],
 [30400, 'AYMARAES', 24570],
 [30500, 'COTABAMBAS', 55208],
 [30600, 'CHINCHEROS', 46544],
 [30700, 'GRAU

In [13]:
poblacion =[]
for x in range(len(poblacion_dict['data'])):
    array = poblacion_dict['data'][x][2]
    poblacion.append(array)
    
poblacion

[63188,
 84672,
 26830,
 51344,
 47827,
 33651,
 119294,
 185276,
 6433,
 13950,
 7710,
 24012,
 50007,
 18496,
 57256,
 8017,
 63264,
 33066,
 56557,
 21787,
 7224,
 24371,
 26675,
 18085,
 474053,
 28630,
 55769,
 120116,
 150758,
 11781,
 24570,
 55208,
 46544,
 21759,
 1175765,
 61708,
 43690,
 34743,
 97458,
 16426,
 54851,
 12797,
 317801,
 32482,
 8341,
 97205,
 75277,
 51838,
 29139,
 9909,
 9292,
 19866,
 17063,
 388170,
 83167,
 83916,
 151714,
 29357,
 123948,
 83913,
 203724,
 145770,
 51678,
 47114,
 22638,
 38602,
 1129854,
 511019,
 24000,
 63131,
 71582,
 34754,
 106476,
 70143,
 62059,
 167910,
 26644,
 47579,
 101735,
 70043,
 121265,
 37503,
 53901,
 14588,
 33883,
 18182,
 85995,
 315799,
 53247,
 32427,
 16372,
 52095,
 138275,
 29160,
 50086,
 36987,
 17114,
 18705,
 445752,
 262110,
 78472,
 14832,
 174016,
 595183,
 59138,
 167385,
 88405,
 22757,
 239105,
 91849,
 40041,
 57604,
 1118724,
 123480,
 15982,
 86411,
 30987,
 85091,
 112970,
 85092,
 168670,
 55868

In [14]:
provincia =[]
for x in range(len(poblacion_dict['data'])):
    array = poblacion_dict['data'][x][1]
    provincia.append(array)
    
provincia

['CHACHAPOYAS',
 'BAGUA',
 'BONGARA',
 'CONDORCANQUI',
 'LUYA',
 'RODRIGUEZ DE MENDOZA',
 'UTCUBAMBA',
 'HUARAZ',
 'AIJA',
 'ANTONIO RAYMONDI',
 'ASUNCION',
 'BOLOGNESI',
 'CARHUAZ',
 'CARLOS FERMIN FITZCARRALD',
 'CASMA',
 'CORONGO',
 'HUARI',
 'HUARMEY',
 'HUAYLAS',
 'MARISCAL LUZURIAGA',
 'OCROS',
 'PALLASCA',
 'POMABAMBA',
 'RECUAY',
 'SANTA',
 'SIHUAS',
 'YUNGAY',
 'ABANCAY',
 'ANDAHUAYLAS',
 'ANTABAMBA',
 'AYMARAES',
 'COTABAMBAS',
 'CHINCHEROS',
 'GRAU',
 'AREQUIPA',
 'CAMANA',
 'CARAVELI',
 'CASTILLA',
 'CAYLLOMA',
 'CONDESUYOS',
 'ISLAY',
 'LA UNION',
 'HUAMANGA',
 'CANGALLO',
 'HUANCA SANCOS',
 'HUANTA',
 'LA MAR',
 'LUCANAS',
 'PARINACOCHAS',
 'PAUCAR DEL SARA SARA',
 'SUCRE',
 'VICTOR FAJARDO',
 'VILCAS HUAMAN',
 'CAJAMARCA',
 'CAJABAMBA',
 'CELENDIN',
 'CHOTA',
 'CONTUMAZA',
 'CUTERVO',
 'HUALGAYOC',
 'JAEN',
 'SAN IGNACIO',
 'SAN MARCOS',
 'SAN MIGUEL',
 'SAN PABLO',
 'SANTA CRUZ',
 'CALLAO',
 'CUSCO',
 'ACOMAYO',
 'ANTA',
 'CALCA',
 'CANAS',
 'CANCHIS',
 'CHUMBIVILCAS',


In [15]:
res = {provincia[i]: poblacion[i] for i in range(len(provincia))}
res

{'CHACHAPOYAS': 63188,
 'BAGUA': 84672,
 'BONGARA': 26830,
 'CONDORCANQUI': 51344,
 'LUYA': 47827,
 'RODRIGUEZ DE MENDOZA': 33651,
 'UTCUBAMBA': 119294,
 'HUARAZ': 185276,
 'AIJA': 6433,
 'ANTONIO RAYMONDI': 13950,
 'ASUNCION': 7710,
 'BOLOGNESI': 24012,
 'CARHUAZ': 50007,
 'CARLOS FERMIN FITZCARRALD': 18496,
 'CASMA': 57256,
 'CORONGO': 8017,
 'HUARI': 63264,
 'HUARMEY': 33066,
 'HUAYLAS': 56557,
 'MARISCAL LUZURIAGA': 21787,
 'OCROS': 7224,
 'PALLASCA': 24371,
 'POMABAMBA': 26675,
 'RECUAY': 18085,
 'SANTA': 474053,
 'SIHUAS': 28630,
 'YUNGAY': 55769,
 'ABANCAY': 120116,
 'ANDAHUAYLAS': 150758,
 'ANTABAMBA': 11781,
 'AYMARAES': 24570,
 'COTABAMBAS': 55208,
 'CHINCHEROS': 46544,
 'GRAU': 21759,
 'AREQUIPA': 1175765,
 'CAMANA': 61708,
 'CARAVELI': 43690,
 'CASTILLA': 34743,
 'CAYLLOMA': 97458,
 'CONDESUYOS': 16426,
 'ISLAY': 54851,
 'LA UNION': 12797,
 'HUAMANGA': 317801,
 'CANGALLO': 32482,
 'HUANCA SANCOS': 8341,
 'HUANTA': 97205,
 'LA MAR': 75277,
 'LUCANAS': 51838,
 'PARINACOCHAS':

In [16]:
test['POBLACION'] = test['PROVINCIA'].map(res)

In [17]:
null_columns = test.columns[test.isnull().any()]
test[test["SEXO"].isnull()][null_columns]

,DEPARTAMENTO,PROVINCIA,DISTRITO,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
2589308,LIMA,LIMA,LIMA,0.0,NaN,2021-03-13,150101.0,10881464.0,1600339.0,9674755.0,15057.0


In [18]:
#find_provincia = test['PROVINCIA'] == 'UCAYALI'
find_distrito = test['DEPARTAMENTO'] == 'LORETO'
find_fecha = test['FECHA_RESULTADO'] == '2022-01-02'

data_exploratoria = test[find_distrito & find_fecha]
data_exploratoria

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
128718,20220311,LORETO,MAYNAS,PUNCHANA,AG,27.0,MASCULINO,2022-01-02,160108.0,19990922.0,57764.0,550551.0,5621.0
247278,20220311,LORETO,MAYNAS,SAN JUAN BAUTISTA,PR,32.0,FEMENINO,2022-01-02,160113.0,20925980.0,57764.0,550551.0,5621.0
297871,20220311,LORETO,MAYNAS,PUNCHANA,PR,31.0,MASCULINO,2022-01-02,160108.0,22425039.0,57764.0,550551.0,5621.0
1109938,20220311,LORETO,MAYNAS,IQUITOS,AG,24.0,MASCULINO,2022-01-02,160101.0,34702609.0,57764.0,550551.0,5621.0
1784980,20220311,LORETO,MAYNAS,IQUITOS,AG,69.0,FEMENINO,2022-01-02,160101.0,3428514.0,57764.0,550551.0,5621.0


In [19]:
data_exploratoria.groupby("SEXO").count()

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
SEXO,,,,,,,,,,,,
FEMENINO,2,2,2,2,2,2,2,2,2,2,2,2
MASCULINO,3,3,3,3,3,3,3,3,3,3,3,3


In [20]:
test['SEXO'] = test['SEXO'].fillna('FEMENINO')

In [21]:
null_columns = test.columns[test.isnull().any()]
test[test["PROVINCIA"].isnull()][null_columns]

,DEPARTAMENTO,PROVINCIA,DISTRITO,EDAD,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
21,LIMA,NaN,NaN,54.0,2020-11-21,NaN,24615020.0,1600339.0,NaN,15057.0
29,LIMA,NaN,NaN,39.0,2020-07-24,NaN,24833739.0,1600339.0,NaN,15057.0
36,LIMA,NaN,NaN,28.0,2022-01-21,NaN,24833787.0,1600339.0,NaN,15057.0
49,ICA,NaN,NaN,22.0,2022-01-15,NaN,24662193.0,103224.0,NaN,10585.0
98,LIMA,NaN,NaN,27.0,2021-02-26,NaN,24769312.0,1600339.0,NaN,15057.0
...,...,...,...,...,...,...,...,...,...,...
3535581,ICA,NaN,NaN,43.0,2022-01-23,NaN,NaN,103224.0,NaN,10585.0
3535669,LIMA,NaN,NaN,23.0,2022-01-11,NaN,NaN,1600339.0,NaN,15057.0
3535693,LIMA,NaN,NaN,0.0,2022-01-21,NaN,NaN,1600339.0,NaN,15057.0
3535701,LIMA,NaN,NaN,70.0,2021-02-28,NaN,NaN,1600339.0,NaN,15057.0


In [22]:
test['FECHA_RESULTADO'] = pd.to_datetime(test['FECHA_RESULTADO'], errors='coerce', dayfirst=True)
test['FECHA_RESULTADO']

0         2021-04-14
1         2021-04-13
2         2022-02-22
3         2022-01-14
4         2020-07-04
             ...    
3535720   2022-01-08
3535721   2020-10-02
3535722   2021-05-29
3535723   2021-03-24
3535724   2022-02-09
Name: FECHA_RESULTADO, Length: 3535725, dtype: datetime64[ns]

In [23]:
test['POBLACION_CIENMIL'] = test['POBLACION']/100000

#test = test.drop(labels="POBLACION_CIENMIL", axis=1)
test

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE,POBLACION_CIENMIL
0,20220311,JUNIN,HUANCAYO,EL TAMBO,AG,68.0,FEMENINO,2021-04-14,120114.0,24615058.0,125271.0,595183.0,9201.0,5.95183
1,20220311,LIMA,LIMA,SAN JUAN DE MIRAFLORES,AG,21.0,MASCULINO,2021-04-13,150133.0,24731649.0,1600339.0,9674755.0,15057.0,96.74755
2,20220311,AREQUIPA,AREQUIPA,PAUCARPATA,AG,27.0,MASCULINO,2022-02-22,40112.0,24731664.0,211730.0,1175765.0,14139.0,11.75765
3,20220311,PIURA,PIURA,CASTILLA,AG,20.0,MASCULINO,2022-01-14,200104.0,24731665.0,157210.0,894847.0,7676.0,8.94847
4,20220311,ANCASH,SANTA,CHIMBOTE,PR,23.0,MASCULINO,2020-07-04,21801.0,24856166.0,124901.0,474053.0,10579.0,4.74053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535720,20220311,LIMA,LIMA,LIMA,PCR,23.0,FEMENINO,2022-01-08,150101.0,NaN,1600339.0,9674755.0,15057.0,96.74755
3535721,20220311,LIMA,LIMA,RIMAC,PR,51.0,FEMENINO,2020-10-02,150128.0,NaN,1600339.0,9674755.0,15057.0,96.74755
3535722,20220311,TACNA,TACNA,CALANA,PCR,37.0,FEMENINO,2021-05-29,230103.0,NaN,53615.0,346192.0,14452.0,3.46192
3535723,20220311,LIMA,LIMA,CHORRILLOS,PCR,68.0,FEMENINO,2021-03-24,150108.0,NaN,1600339.0,9674755.0,15057.0,96.74755


In [24]:
salidasxsemanas = test.sort_values(by = 'FECHA_RESULTADO')
start_date = "2022-01-16"
end_date = "2022-03-06"

after_start_date = salidasxsemanas["FECHA_RESULTADO"] >= start_date
before_end_date = salidasxsemanas["FECHA_RESULTADO"] <= end_date
between_two_dates = after_start_date & before_end_date

filtered_dates = salidasxsemanas.loc[between_two_dates]

filtered_dates

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE,POBLACION_CIENMIL
2018562,20220311,LIMA,NaN,NaN,AG,87.0,MASCULINO,2022-01-16,NaN,5471760.0,1600339.0,NaN,15057.0,NaN
2753223,20220311,PUNO,PUNO,PUNO,AG,43.0,FEMENINO,2022-01-16,210101.0,12821986.0,65175.0,230219.0,5265.0,2.30219
1940711,20220311,LIMA,LIMA,SAN LUIS,PCR,51.0,FEMENINO,2022-01-16,150134.0,5064677.0,1600339.0,9674755.0,15057.0,96.74755
2696631,20220311,CAJAMARCA,CAJAMARCA,CAJAMARCA,PCR,74.0,FEMENINO,2022-01-16,60101.0,12126964.0,95474.0,388170.0,6568.0,3.88170
2087713,20220311,LIMA,LIMA,LOS OLIVOS,AG,28.0,FEMENINO,2022-01-16,150117.0,5797874.0,1600339.0,9674755.0,15057.0,96.74755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168245,20220311,AMAZONAS,RODRIGUEZ DE MENDOZA,SAN NICOLAS,AG,7.0,MASCULINO,2022-03-06,10601.0,35484846.0,43766.0,33651.0,10254.0,0.33651
685793,20220311,LIMA,HUAURA,HUALMAY,AG,27.0,MASCULINO,2022-03-06,150805.0,27551979.0,1600339.0,240717.0,15057.0,2.40717
2562384,20220311,LAMBAYEQUE,CHICLAYO,CHICLAYO,PCR,57.0,FEMENINO,2022-03-06,140101.0,10700689.0,105224.0,862709.0,8028.0,8.62709
3047739,20220311,LAMBAYEQUE,CHICLAYO,CHICLAYO,PCR,56.0,FEMENINO,2022-03-06,140101.0,15558381.0,105224.0,862709.0,8028.0,8.62709


In [25]:
filtered_dates['PROVINCIA'] = filtered_dates['PROVINCIA'].replace('EN INVESTIGACIÃN', np.nan)
filtered_dates

<ipython-input-25-bdb0c20004fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dates['PROVINCIA'] = filtered_dates['PROVINCIA'].replace('EN INVESTIGACIÃN', np.nan)


,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE,POBLACION_CIENMIL
2018562,20220311,LIMA,NaN,NaN,AG,87.0,MASCULINO,2022-01-16,NaN,5471760.0,1600339.0,NaN,15057.0,NaN
2753223,20220311,PUNO,PUNO,PUNO,AG,43.0,FEMENINO,2022-01-16,210101.0,12821986.0,65175.0,230219.0,5265.0,2.30219
1940711,20220311,LIMA,LIMA,SAN LUIS,PCR,51.0,FEMENINO,2022-01-16,150134.0,5064677.0,1600339.0,9674755.0,15057.0,96.74755
2696631,20220311,CAJAMARCA,CAJAMARCA,CAJAMARCA,PCR,74.0,FEMENINO,2022-01-16,60101.0,12126964.0,95474.0,388170.0,6568.0,3.88170
2087713,20220311,LIMA,LIMA,LOS OLIVOS,AG,28.0,FEMENINO,2022-01-16,150117.0,5797874.0,1600339.0,9674755.0,15057.0,96.74755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168245,20220311,AMAZONAS,RODRIGUEZ DE MENDOZA,SAN NICOLAS,AG,7.0,MASCULINO,2022-03-06,10601.0,35484846.0,43766.0,33651.0,10254.0,0.33651
685793,20220311,LIMA,HUAURA,HUALMAY,AG,27.0,MASCULINO,2022-03-06,150805.0,27551979.0,1600339.0,240717.0,15057.0,2.40717
2562384,20220311,LAMBAYEQUE,CHICLAYO,CHICLAYO,PCR,57.0,FEMENINO,2022-03-06,140101.0,10700689.0,105224.0,862709.0,8028.0,8.62709
3047739,20220311,LAMBAYEQUE,CHICLAYO,CHICLAYO,PCR,56.0,FEMENINO,2022-03-06,140101.0,15558381.0,105224.0,862709.0,8028.0,8.62709


In [26]:
filtered_dates.isnull().sum()

FECHA_CORTE              0
DEPARTAMENTO           321
PROVINCIA            37075
DISTRITO             37075
METODODX_x               0
EDAD                     0
SEXO                     0
FECHA_RESULTADO          0
UBIGEO               37075
id_persona           17873
METODODX_y             322
POBLACION            38669
INDICE                 322
POBLACION_CIENMIL    38669
dtype: int64

In [27]:
weekly_sales = filtered_dates.groupby(["SEXO","DEPARTAMENTO","PROVINCIA", "POBLACION", "POBLACION_CIENMIL", "INDICE", pd.Grouper(key="FECHA_RESULTADO",freq="W-SUN")]).size()
weekly_sales = weekly_sales.unstack(0).fillna(0)
weekly_sales.sort_values(by=['FECHA_RESULTADO'], inplace=True, ascending=True)
weekly_sales.loc[:,'TOTAL'] = weekly_sales.sum(numeric_only=True, axis=1)

weekly_sales = weekly_sales.reset_index()
weekly_sales

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
0,AMAZONAS,BAGUA,84672.0,0.84672,10254.0,2022-01-16,2.0,0.0,2.0
1,CAJAMARCA,CAJABAMBA,83167.0,0.83167,6568.0,2022-01-16,6.0,5.0,11.0
2,PIURA,HUANCABAMBA,118533.0,1.18533,7676.0,2022-01-16,4.0,4.0,8.0
3,AYACUCHO,VICTOR FAJARDO,19866.0,0.19866,7175.0,2022-01-16,0.0,1.0,1.0
4,PIURA,MORROPON,173193.0,1.73193,7676.0,2022-01-16,31.0,32.0,63.0
...,...,...,...,...,...,...,...,...,...
1491,LIMA,HUARAL,194375.0,1.94375,15057.0,2022-03-06,19.0,14.0,33.0
1492,ANCASH,PALLASCA,24371.0,0.24371,10579.0,2022-03-06,2.0,2.0,4.0
1493,MOQUEGUA,MARISCAL NIETO,95551.0,0.95551,24995.0,2022-03-06,25.0,41.0,66.0
1494,AREQUIPA,CAYLLOMA,97458.0,0.97458,14139.0,2022-03-06,10.0,4.0,14.0


In [28]:
weekly_sales.groupby("FECHA_RESULTADO").count()

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FEMENINO,MASCULINO,TOTAL
FECHA_RESULTADO,,,,,,,,
2022-01-16,181,181,181,181,181,181,181,181
2022-01-23,193,193,193,193,193,193,193,193
2022-01-30,193,193,193,193,193,193,193,193
2022-02-06,193,193,193,193,193,193,193,193
2022-02-13,192,192,192,192,192,192,192,192
2022-02-20,190,190,190,190,190,190,190,190
2022-02-27,180,180,180,180,180,180,180,180
2022-03-06,174,174,174,174,174,174,174,174


In [ ]:
cero_cases = weekly_sales['PROVINCIA']== "ATALAYA"
data_cero = weekly_sales[cero_cases]
data_cero

In [ ]:
weekly_sales[['FEMENINO','MASCULINO', 'TOTAL']] = weekly_sales[['FEMENINO','MASCULINO', 'TOTAL']].div(weekly_sales['POBLACION_CIENMIL'].values,axis=0)
weekly_sales

In [ ]:
weekly_sales.FEMENINO = weekly_sales.FEMENINO.round()
weekly_sales.MASCULINO = weekly_sales.MASCULINO.round()
weekly_sales.TOTAL = weekly_sales.FEMENINO + weekly_sales.MASCULINO
weekly_sales.FECHA_RESULTADO = weekly_sales.FECHA_RESULTADO.dt.strftime('%Y-%m-%d')
weekly_sales

In [ ]:
weekly_sales.to_csv('dataset_covid_total.csv' , index=False)

In [ ]:
weekly_sales.to_json('dataset_covid_total.json', orient="table")